# azureの使用方法関数一覧
- pipインストールは起動後に毎回必要
- ※後日キーの置き換え

## DB周り

#### azureのai searchのインデックス一覧取得

In [17]:
%pip install azure-core azure-search-documents


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient

service_name = "maps"
admin_key = "EKz0IbLfgliPcYLCYcQVLiukrwthFaIcNe4byooh1mAzSeAuV0Vp"
credential = AzureKeyCredential(admin_key)
endpoint = f"https://{service_name}.search.windows.net/"

# SearchIndexClientを作成
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

# インデックスの一覧を取得
indexes = index_client.list_indexes()

# インデックス名を表示
for index in indexes:
    print(f"Index Name: {index.name}")

Index Name: 10files-test
Index Name: azureblob-index
Index Name: patent-index
Index Name: vector-1725588957492


#### azureのストレージ一覧取得

In [13]:
%pip install azure-storage-blob azure-identity pymupdf


  Obtaining dependency information for pymupdf from https://files.pythonhosted.org/packages/38/80/f8d8ae555b237574005faef8a181a5c6a1d983e16a982b65ccc56a42faa2/PyMuPDF-1.24.13-cp39-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.2 MB 217.9 kB/s eta 0:01:15
   ---------------------------------------- 0.0/16.2 MB 217.9 kB/s eta 0:01:15
   ---------------------------------------- 0.0/16.2 MB 163.8 kB/s eta 0:01:39
   ---------------------------------------- 0.0/16.2 MB 178.6 kB/s eta 0:01:31
   ---------------------------------------- 0.1/16.2 MB 308.0 kB/s eta 0:00:53
   - -------------------------------------- 0.6/16.2 MB 1.9 MB/s eta 0:00:09
   --- ------------------------------------ 1.3/16.2 MB 3.8 MB/s eta 0:00:04
   --- -------------------------


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from azure.storage.blob import BlobServiceClient

# 接続文字列を設定
connection_string = "DefaultEndpointsProtocol=https;AccountName=maps5370605259;AccountKey=jA9t0HLlnYSLmPbSf6RkEg8emdkjhHO9JfkL5XT8HNX8H4kfq+WCHO/v0fwXgN4b083JhPDP77Fk+AStBofZPg==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# # コンテナ一覧の取得
def get_storage_nameList():
    containers = blob_service_client.list_containers()
    for container in containers:
        print("Container Name:", container['name'])

get_storage_nameList()

Container Name: ateam-testcontainer-test-10files
Container Name: azureml
Container Name: azureml-blobstore-7e664d9f-7a26-47bc-81b8-b1545cc9cedd
Container Name: insights-logs-auditevent
Container Name: insights-metrics-pt1m
Container Name: revisions
Container Name: snapshots
Container Name: snapshotzips


#### azureのストレージからランダムに請求項の取得

In [ ]:
%pip install azure-storage-blob azure-identity pymupdf

In [ ]:
from azure.storage.blob import BlobServiceClient
import random
import fitz

# 接続文字列を設定
connection_string = "DefaultEndpointsProtocol=https;AccountName=maps5370605259;AccountKey=jA9t0HLlnYSLmPbSf6RkEg8emdkjhHO9JfkL5XT8HNX8H4kfq+WCHO/v0fwXgN4b083JhPDP77Fk+AStBofZPg==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# コンテナの指定
container_name = "azureml-blobstore-7e664d9f-7a26-47bc-81b8-b1545cc9cedd"
container_client = blob_service_client.get_container_client(container_name)


# 選択したPDFの内容を表示
def get_random_nameList(count:int=3) -> list[dict] :
    # BLOB一覧の取得
    blobs = list(container_client.list_blobs())
    random_blobs = random.sample(blobs, count)
    result = []
    for pdf_blob in random_blobs:
        # BLOBデータのダウンロード
        blob_client = container_client.get_blob_client(pdf_blob.name)
        pdf_data = blob_client.download_blob().readall()

        # PDFデータの読み取り
        with fitz.open(stream=pdf_data, filetype="pdf") as pdf_document:
            # 開始と終了のキーワード
            start_keyword = "【請求項"
            end_keyword = "【発明の詳細な説明】"
            for page_number in range(pdf_document.page_count):
                page = pdf_document.load_page(page_number)
                text = page.get_text()

                # キーワードによる範囲抽出
                start_index = text.find(start_keyword)
                end_index = text.find(end_keyword)
                if start_index != -1:
                    # 最初のキーワードが見つかった場合、次のキーワードを探す
                    next_claim_index = text.find(start_keyword, start_index + len(start_keyword))
                    end_desc_index = text.find(end_keyword, start_index + len(start_keyword))

                    # 2つのインデックスのうち、存在する方か、より早く出現する方を終了位置とする
                    if next_claim_index != -1 and (end_desc_index == -1 or next_claim_index < end_desc_index):
                        end_index = next_claim_index
                    else:
                        end_index = end_desc_index

                if start_index != -1 and end_index != -1 and end_index > start_index:
                    # 該当部分の抽出と表示
                    relevant_text = text[start_index:end_index]
                    result.append({
                        "title": pdf_blob.name,
                        "page":page_number + 1,
                        "text": relevant_text
                    })
                    break
                elif start_index != -1:
                    # 終了キーワードが同ページにない場合、開始以降をすべて表示
                    result.append({
                        "title": pdf_blob.name,
                        "page":page_number + 1,
                        "text": text[start_index:]
                    })
                    break
    return result

get_random_nameList()


## azureのai活用

#### 検索以外のサンプル

##### 純粋なベクトル化

In [ ]:
%pip install openai

In [ ]:
from openai import AzureOpenAI

# Azure OpenAI APIの設定
client = AzureOpenAI(
    api_version="2023-03-15-preview",
    azure_endpoint="https://ateamopenai.openai.azure.com/",
    api_key="b7e21555bf544a038f1b26c9d9b0fb9e",
)
def get_embedding(text: str, engine="text-embedding-ada-002"):
    # OpenAI埋め込みモデルを使用してテキストをベクトル化
    response = client.embeddings.create(
        input=text,
        model=engine  # 埋め込みモデルの名前
    )
    return response.data[0].embedding

# クエリをベクトル化
query = "高速計算の簡略化"
query_vector = get_embedding(query)

print(f"Query Vector: {query_vector}")



Query Vector: [-0.01790501, 0.018059121, 0.008483164, -0.02894503, -0.0020384812, 0.006539251, -0.014626628, -0.028622797, -0.0057511786, -0.047158264, 0.011768551, 0.015327137, 0.0035270627, 0.014668658, -0.014444496, -0.010493625, 0.014402465, -0.022472328, 0.022332227, -0.013008452, 0.005218792, 0.023803296, -0.020174658, -0.026381169, -0.031550925, 0.0031873158, 0.021743799, -0.010353522, 0.010367533, -0.021309484, 0.034437023, -0.020693036, -0.00980012, -0.021071311, 0.0036391441, -0.0041154902, 0.0031838133, -0.004612852, 0.023607153, -0.001063898, 0.0295895, 0.0066863582, 0.010010273, 0.0070226025, 0.006164479, -0.0010236187, 0.007621538, -0.01015738, -0.015537289, 0.015060944, 0.0013248376, 0.00973007, -0.00041702174, -0.0056601125, -0.0009264231, 0.015439218, -0.0072782882, 0.0046373694, -0.004182039, 0.003367697, 0.032195393, 0.011236164, -0.025820762, 0.01165647, -0.0088474285, -0.01475272, -0.0023239385, -0.015985616, -0.016097696, 0.008784383, 0.0025848781, 0.013757996, -0

In [ ]:
%pip install openai azure-search-documents

##### 純粋なOpenAIの通信

In [44]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version="2023-03-15-preview",
    azure_endpoint="https://ateamopenai.openai.azure.com/",
    api_key="b7e21555bf544a038f1b26c9d9b0fb9e",
)
def chat_sample(message: str) -> str:
    completion = client.chat.completions.create(
        model="gpt-35-turbo", 
        messages=[
            {
                "role": "user",
                "content": message,
            },
        ],
    )
    return(completion.choices[0].message.content)

chat_sample("特許の見方について教えて")

'特許は、発明を保護するための法的文書です。特許文書には、以下の情報が含まれています。\n\n1. 発明の名称\n2. 発明者の名前\n3. 発明の概要\n4. 発明を実現するための方法、手段、装置などの詳細な説明\n5. 発明の利点や効果に関する説明\n6. 発明に対する権利範囲の明確な記述\n\n特許文書をより詳しく理解するためには、以下の点に注目することが重要です。\n\n1. 特許の請求項：特許の請求項には、発明に対する保護範囲が明確に記載されています。請求項は、特許侵害の判断に非常に重要な役割を果たします。\n\n2. 図面：多くの特許文書には、発明を説明する図面が含まれています。図面を見ることで、発明の構造や機能をより詳しく理解することができます。\n\n3. 引用文献：特許文書には、その発明が先行して公知されている技術や知見に関する情報が含まれています。これらの情報を確認することで、発明の新規性や非自明性が評価できます。'

#### azureのai searchの検索
[公式](https://learn.microsoft.com/en-us/python/api/azure-search-documents/azure.search.documents.searchclient?view=azure-python)

##### vector

In [42]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

service_name = "maps"
admin_key = "EKz0IbLfgliPcYLCYcQVLiukrwthFaIcNe4byooh1mAzSeAuV0Vp"
credential = AzureKeyCredential(admin_key)
endpoint = f"https://{service_name}.search.windows.net/"
index_name="vector-1725588957492"

def search_sample_vector(message: str,top:int=5) -> str:
        search_client = SearchClient(endpoint, index_name, credential)
        # ベクトル検索を実行
        vector_query = VectorizedQuery(
                vector=get_embedding(message),
                fields="text_vector"
                )
        return search_client.search(
                search_text="",# フルテキスト検索は空にします
                vector_queries=[vector_query],
                top=top 
        )


for result in search_sample_vector("電卓の高速化"):
    print("chunk:",result.get("chunk"))

chunk: ル対象を借用するために前記ユーザの携帯端末のコンピュータによって実行されるプログ
ラムであって、
前記レンタル対象管理システムは、管理サーバを含み、
前記携帯端末は、複数の機能として、
前記携帯端末が、前記いずれかのステーションにおいて、前記複数のレンタル対象のうち
前記ユーザが事前の予約なしで現地で自主的に選択したいずれかのレンタル対象に対応す
るコードを取得する機能と、
前記携帯端末が、前記取得されたコードまたはそのコードが変換されたものを前記いずれ
かのレンタル対象に対応するレンタル対象ＩＤとして前記管理サーバに送信する機能と、
前記ユーザが前記携帯端末に貸出リクエストを入力した場合に、その貸出リクエストを前
記管理サーバに送信する機能と
を含み、
前記管理サーバは、前記携帯端末から前記貸出リクエストと前記レンタル対象ＩＤとを受
信した場合に、前記ユーザへの前記いずれかのレンタル対象の貸出しを許可し、
当該プログラムは、前記複数の機能を実現するために前記コンピュータによって実行され
るプログラム。
【請求項８】
前記複数の機能は、さらに、
前記ユーザによる前記いずれかのレンタル対象の使用を禁止するロック状態にあるロック
装置について前記管理サーバから解錠信号を受信する機能と、
その受信した解錠信号を前記ロック装置に送信する機能と
を含む請求項７に記載のプログラム。
【請求項９】
請求項６に記載のプログラムをコンピュータ読み取り可能に記録した記録媒体。
【請求項１０】
請求項７または８に記載のプログラムをコンピュータ読み取り可能に記録した記録媒体。


	AmendmentHeader
	WrittenAmendmentBag
chunk: 値に基づいて、導出部が、前記カテゴリ毎についての他の間接排出量を導出する段階と
を備える、導出方法。
【請求項５】
事業者より直接的に排出された温室効果ガスの排出量を示す直接排出量、及び事業者がエ
ネルギーを購入することで間接的に排出された温室効果ガスの排出量を示す間接排出量に
含まれない事業者の活動により排出された温室効果ガスの排出量を示す他の間接排出量を
導出する導出装置としてコンピュータを機能させるためのプログラムであって、
主事業者と、前記主事業者の活動に関わる他の事業者及び他の事業者の業種とを関連づけ

##### index

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

# 必要な情報を設定
service_name = "maps"
admin_key = "EKz0IbLfgliPcYLCYcQVLiukrwthFaIcNe4byooh1mAzSeAuV0Vp"
credential = AzureKeyCredential(admin_key)
endpoint = f"https://{service_name}.search.windows.net/"
index_name="vector-1725588957492"


def search_sample_index(message: str,top:int=5) -> str:
    client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

    # クエリを実行して検索
    query = message
    return client.search(search_text=query,top=top )


for result in search_sample_index("電卓の高速化"):
    print("chunk:",result.get("chunk"))


##### hybrid

In [20]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

service_name = "maps"
admin_key = "EKz0IbLfgliPcYLCYcQVLiukrwthFaIcNe4byooh1mAzSeAuV0Vp"
credential = AzureKeyCredential(admin_key)
endpoint = f"https://{service_name}.search.windows.net/"
index_name="vector-1725588957492"

def search_sample_hybrid(message: str,top:int=5) -> str:
        search_client = SearchClient(endpoint, index_name, credential)
        # ベクトル検索を実行
        vector_query = VectorizedQuery(
                vector=get_embedding(message),
                fields="text_vector",
                top_k=5,  # 類似度の高い上位5件を取得
                boost=1.0  # ベクトル検索の重要度（boost値）
                )
        return search_client.search(
                search_text=message,
                vector_queries=[vector_query],
                top=top 
        )


for result in search_sample_hybrid("電卓の高速化"):
    print("chunk:",result.get("chunk"))

top_k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizedQuery'> and will be ignored
boost is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizedQuery'> and will be ignored


chunk: を正確に行うことができるとともに、あとから改ざんすることが非常に困難になり、記録
としての信頼性を向上させることができる。その結果、交換所における景品の管理におい

10

20

30

40

50



(22) JP　2024-105482　A　2024.8.6

て、正確な管理と、直感的な管理の両方を実現することができる。
【０１０８】
さらに、本実施形態の景品管理方法において、上述したように、景品の種類と個数に応じ
たデータをランダム配列の時刻データにリンクさせる処理を実行すると、景品の内容（景
品の種類と個数に応じたデータ）を変更して景品を抜き取って（または、金庫またはレジ
の現金を抜き取って）、１日の合計・集計データの帳尻をあわせることをできなくするこ
とができる。その場合、ランダム配列のデータにリンクさせる処理をすることで、不正防
止の抑止効果を得ることができ、言い換えると、改ざんの抑止効果を高めることができる
。
【０１０９】
次に、図９および図１０（ａ）及び（ｂ）は、本実施形態の電卓装置（景品電卓）１０１
の構成の一例を示す図である。図９は、電卓装置（景品電卓）１０１の構成を模式的に示
す斜視図である。図１０（ａ）および（ｂ）は、それぞれ、電卓装置（景品電卓）１０１
の構成を模式的に示す上面図および側面図である。
【０１１０】
あくまで一例であるが、図１０（ａ）において、全幅Ｗ１は１７０ｍｍ（±１０ｍｍ）、
景品電卓１０１の右端から景品選択ボタン（幅広ボタン）６９の右端までの幅Ｗ２は１１
６ｍｍ（±１０ｍｍ）、ディスプレイ６５の中央と入力部６３の中央の間隔Ｄは１２５ｍ
ｍ（±１０ｍｍ）である。図１０（ｂ）において、高さＨは１１４ｍｍ（±１０ｍｍ）、
長さ１は２９０ｍｍ（±１０ｍｍ）、長さ２は２６９．６９ｍｍ（±１０ｍｍ）である。
なお、実施形態の電卓装置（景品電卓）１０１の構成・構造は、例示したものに限定され
ず、適宜好適なものを設計・選択することが可能である。なお、本実施形態の電卓装置（
景品電卓）１０１に、交換所で発生する手数料（例えば、金銭の１％（または、２％、３
％）の手数料）を計算する機能を設けても構わない。
【０１１１】
以上、本発明を好適な実施形態により説明してきたが、こうした記述は限定事項ではなく
、勿論、種々の改

##### semantic
- 検索エンジンが単純なキーワードの一致検索だけではなく、ユーザの意図やクエリの意味を理解して、関連性の高い情報を提供

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

service_name = "maps"
admin_key = "EKz0IbLfgliPcYLCYcQVLiukrwthFaIcNe4byooh1mAzSeAuV0Vp"
credential = AzureKeyCredential(admin_key)
endpoint = f"https://{service_name}.search.windows.net/"
index_name="vector-1725588957492"

def search_sample_semantic(message: str,top:int=5) -> str:
        search_client = SearchClient(endpoint, index_name, credential)

        return search_client.search(
                search_text=message,
                query_type="semantic", 
                query_caption="extractive",  # 説明を含む結果を提供する設定
                query_answer="extractive",  # クエリに基づく要約回答を取得する
                top=top 
        )


for result in search_sample_semantic("電卓の高速化"):
    print("chunk:",result.get("chunk"))

chunk: に送信して記録する機能とを備え、中央演算処理装置は、分割されたデータを時刻データ
に紐付ける処理を行う機能を有しているので、遊技場で使用される景品の管理を行うこと
ができる電卓装置を提供することができる。また、時刻データに紐付ける処理を行う機能
が、分割されたデータのそれぞれを、ランダム配列の時刻データにリンクさせる処理を行
うものであることにより、景品の内容（景品の種類と個数に応じたデータ）を変更して景
品を抜き取って（または、金庫またはレジの現金を抜き取って）１日の合計・集計データ
の帳尻をあわせることが困難となり、その結果、不正防止の抑止効果を得ることができる
。
【図面の簡単な説明】
【００４６】
【図１】本発明の実施形態に係る換金管理装置１００を含む景品換金システムを説明する
ブロック図である。
【図２】本発明の実施形態に係る換金管理装置１００の構成を説明する図である。
【図３】本発明の実施形態に係る換金管理装置１００の構造を模式的に示す斜視図である
。
【図４】本発明の実施形態に係る換金管理装置１００の構造を模式的に示す斜視図である
。
【図５】本発明の実施形態に係る換金管理装置１００に使用する電源３０を示す図である
。
【図６】本発明の実施形態に係る換金管理装置１００を含む景品換金システムを説明する
図である。
【図７】本発明の実施形態に係る電卓装置（景品電卓）１０１の構成を説明する図である
。
【図８】本発明の実施形態に係る景品の管理方法を説明するためのフローチャートである
。
【図９】本発明の実施形態に係る電卓装置（景品電卓）１０１の構成を模式的に示す斜視
図である。
【図１０】（ａ）および（ｂ）は、それぞれ、本発明の実施形態に係る電卓装置（景品電
卓）１０１の構成を模式的に示す上面図および側面図である。
【図１１】本発明の実施形態に係る換金管理プログラムをコンピュータに実行させたとき
の入力画面である。
【図１２】本発明の実施形態に係る換金管理プログラムをコンピュータに実行させたとき
の入力後の処理実行画面（登録画面）である。
【図１３】本発明の実施形態に係る換金管理プログラムをコンピュータに実行させたとき
の処理実行後の結果画面（出力画面）である。
【発明を実施するための形態】
【００４７】
以下、図面を参照しながら、本発明の好適

## RAG実装

#### RAG(ai searchの検索使用)

In [1]:
%pip install openai azure-search-documents azure-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from openai import AzureOpenAI


client = AzureOpenAI(
    api_version="2023-03-15-preview",
    azure_endpoint="https://ateamopenai.openai.azure.com/",
    api_key="b7e21555bf544a038f1b26c9d9b0fb9e",
)

def chat_RAG_sample(query: str) -> str:
    # 検索結果を取得
    search_results = search_sample_vector(query,1)
    # 各種選択可能
    #search_sample_vector(query,1)
    #search_sample_index(query,1)
    #search_sample_hybrid(query,1)
    #search_sample_semantic(query,1)
    context = "\n".join([result["chunk"] for result in search_results])
    
    completion = client.completions.create(
        model="gpt-35-turbo", #ateam-gpt-4o-miniは未対応
        prompt=f"質問{query}あなたは特許の侵害に詳し弁護士です。以下のドキュメントを用いてuserの質問に答えてください。\n{context}",
        max_tokens=150
    )
    return(completion.choices[0].text)

chat_RAG_sample("電卓の高速化をする方法")

'、\nマウス）、接続回路、インターフェイスなどを備える。コンピュータに、本実施形態に係る\nプログラムを導入した場合、前述の表示装置、入力装置、接続回路、インターフェイス\nなどは、プログラムに適応したものに設計変更できるし、また設計変更することも可能で\nある。なお、導入したプログラムは、コンピュ'

## 検証テストの実装

### ランダムに生成した請求項を取得可能であるか

In [ ]:

def test()->int:
    temp=get_random_nameList()
    # タイトルを保存するリスト
    titles = [claim["title"] for claim in temp] 
    combined_claims = "\n\n".join([f"{claim['text']}" for claim in temp])
    # まとめたテキストを新しい請求項の生成用に加工
    prompt = (
        "以下に複数の請求項を示します。それらを基にして、新しい創造的な請求項を1つ作成してください。:\n\n"
        f"{combined_claims}\n\n"
        "これらを基にした新しい請求項を以下に記述してください。:"
    )
    text = chat_sample(prompt)
    print("titles",titles)
    print(text)
    result_titles=[]
    # 検索の選択
    for result in search_sample_vector(text):
        result_titles.append(result.get("title"))
    print("result_titles",result_titles)
    #一致しているtitleをカウント
    return sum(1 for title in result_titles if title in titles)
test()

titles ['2024105644.pdf', '2024106063.pdf', '2024107238.pdf']
【新しい請求項】
遊技機における特定遊技状態に移行する条件に第１および第２の特別情報の両方を満たすことを特徴とする遊技機。
result_titles ['2024047950.pdf', '2023046195.pdf', '2024047951.pdf', '2023106044.pdf', '2024048448.pdf']


0